<a href="https://colab.research.google.com/github/cyss93/CP1_team8/blob/%EC%98%A4%EC%B0%BD%EC%98%81/Summer_products_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import pandas as pd
import numpy as np
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

from matplotlib import style
import matplotlib.pyplot as plt

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import Ridge


import time
import itertools

In [130]:
df = pd.read_csv('summer-products-with-rating-and-performance_2020-08.csv')

- 변수 정리
 'title' = 
 'title_orig',
 'price',
 'retail_price',
 'currency_buyer',
 'units_sold',
 'uses_ad_boosts',
 'rating',
 'rating_count',
 'rating_five_count',
 'rating_four_count',
 'rating_three_count',
 'rating_two_count',
 'rating_one_count',
 'badges_count',
 'badge_local_product',
 'badge_product_quality',
 'badge_fast_shipping',
 'tags',
 'product_color',
 'product_variation_size_id',
 'product_variation_inventory',
 'shipping_option_name',
 'shipping_option_price',
 'shipping_is_express',
 'countries_shipped_to',
 'inventory_total',
 'has_urgency_banner',
 'urgency_text',
 'origin_country',
 'merchant_title',
 'merchant_name',
 'merchant_info_subtitle',
 'merchant_rating_count',
 'merchant_rating',
 'merchant_id',
 'merchant_has_profile_picture',
 'merchant_profile_picture',
 'product_url',
 'product_picture',
 'product_id',
 'theme',
 'crawl_month'

In [131]:
df.isna().sum()

title                              0
title_orig                         0
price                              0
retail_price                       0
currency_buyer                     0
units_sold                         0
uses_ad_boosts                     0
rating                             0
rating_count                       0
rating_five_count                 45
rating_four_count                 45
rating_three_count                45
rating_two_count                  45
rating_one_count                  45
badges_count                       0
badge_local_product                0
badge_product_quality              0
badge_fast_shipping                0
tags                               0
product_color                     41
product_variation_size_id         14
product_variation_inventory        0
shipping_option_name               0
shipping_option_price              0
shipping_is_express                0
countries_shipped_to               0
inventory_total                    0
h

In [132]:
# 결측값 있는 행 제거
# has_urgency_banner, urgency_text, merchant_profile_picture 이 세 컬럼은 결측값의 비율이 너무 높기때문에 drop
df.drop(columns = ['has_urgency_banner', 'urgency_text', 'merchant_profile_picture'], inplace = True)

In [133]:
# 결측값 있는 행 제거
df.dropna(axis = 0, inplace = True)

In [134]:
df.isna().sum()

title                           0
title_orig                      0
price                           0
retail_price                    0
currency_buyer                  0
units_sold                      0
uses_ad_boosts                  0
rating                          0
rating_count                    0
rating_five_count               0
rating_four_count               0
rating_three_count              0
rating_two_count                0
rating_one_count                0
badges_count                    0
badge_local_product             0
badge_product_quality           0
badge_fast_shipping             0
tags                            0
product_color                   0
product_variation_size_id       0
product_variation_inventory     0
shipping_option_name            0
shipping_option_price           0
shipping_is_express             0
countries_shipped_to            0
inventory_total                 0
origin_country                  0
merchant_title                  0
merchant_name 

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1457 entries, 0 to 1572
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   title                         1457 non-null   object 
 1   title_orig                    1457 non-null   object 
 2   price                         1457 non-null   float64
 3   retail_price                  1457 non-null   int64  
 4   currency_buyer                1457 non-null   object 
 5   units_sold                    1457 non-null   int64  
 6   uses_ad_boosts                1457 non-null   int64  
 7   rating                        1457 non-null   float64
 8   rating_count                  1457 non-null   int64  
 9   rating_five_count             1457 non-null   float64
 10  rating_four_count             1457 non-null   float64
 11  rating_three_count            1457 non-null   float64
 12  rating_two_count              1457 non-null   float64
 13  rat

In [136]:
# 제외 변수들
# 첫번째 제외 변수: title, title_orig, currency_buyer, tags, product_color, product_variation_size_id
# 제외 변수중 고려해볼 만 한 변수
## tags(인코딩 활용. 인코딩 활용시 컬럼이 과하게 늘어난다. 주의), product_variation_size_id
## shipping_option_name: 프랑스어와 아랍어? 등 뒤죽박죽
## urgency_text: 품절임박! 같은 건데, 프랑스어로 되어있다. 일단은 빼고 이후에 원핫인코딩해서 넣어보자 (missing 값이 1100개이다. 유의할 것)
## origin_country: 이것도 원핫인코딩해서 적용 가능
## merchant_title: 판매자 이름 우선 제외
## merchant_name, merchant_id: 위와 동일
## merchant_info_subtitle: 판매자 정보 글. 일단 제외. 배지 여부와 다른 변수로도 판단 가능?
## merchant_profile_picture: 판매자 사진 링크
## product_url, product_picture: 링크 데이터
## product_id: 제품 키
## theme: 해당 제품들을 찾기위해 검색한 검색어. summer로 모두 동일해서 제외
## crawl_month: 크롤링 날짜라서 제외
## has_urgency_banner: 결측값이 많아서 일단 제외 (결측값이 아마 없다는 뜻?) (1000개 이상의 결측값)
## rating_five_count + rating_four_count + rating_three_count + rating_two_count + rating_one_count : 결측값이 많음 (36개)

In [137]:
# 상수항 추가. 실행이 될 때마다 추가되기 때문에 독립된 셀에서 위로 빼두었습니다.
df = sm.add_constant(df,has_constant="add")

In [138]:
# 다중모델 학습을 위한 특성
features = ['price',
            'retail_price',
            'uses_ad_boosts',
            'rating',
            'rating_count',
            'rating_five_count',
            'rating_four_count',
            'rating_three_count',
            'rating_two_count',
            'rating_one_count',
            'badges_count',
            'badge_local_product',
            'badge_product_quality',
            'badge_fast_shipping',
            'product_variation_inventory',
            'shipping_option_price',
            'shipping_is_express',
            'countries_shipped_to',
            'inventory_total',
            'merchant_rating_count',
            'merchant_rating',
            'merchant_has_profile_picture',
            'const']

In [139]:
# ## train/test 데이터 분할
# train = df.sample(frac=0.75,random_state=1)
# test = df.drop(train.index)

# # target 설정
# target = 'units_sold'
# y_train = train[target]
# y_test = test[target]

# # baseline
# # target인 units_sold의 평균으로 일단 선정
# predict = y_train.mean()
# print(predict)

target = 'units_sold'
x = df[features]
y = df[target]

# 분할
X_train ,X_test , y_train , y_test = train_test_split(x,y,train_size=0.7,test_size=0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1019, 23) (438, 23) (1019,) (438,)


In [140]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1019 entries, 977 to 1555
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price                         1019 non-null   float64
 1   retail_price                  1019 non-null   int64  
 2   uses_ad_boosts                1019 non-null   int64  
 3   rating                        1019 non-null   float64
 4   rating_count                  1019 non-null   int64  
 5   rating_five_count             1019 non-null   float64
 6   rating_four_count             1019 non-null   float64
 7   rating_three_count            1019 non-null   float64
 8   rating_two_count              1019 non-null   float64
 9   rating_one_count              1019 non-null   float64
 10  badges_count                  1019 non-null   int64  
 11  badge_local_product           1019 non-null   int64  
 12  badge_product_quality         1019 non-null   int64  
 13  b

In [141]:
#Train the MLR / 회귀모델적합
Model = sm.OLS(y_train, X_train)
FittedModel = Model.fit()

print(FittedModel.summary())

                            OLS Regression Results                            
Dep. Variable:             units_sold   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     283.7
Date:                Mon, 04 Jul 2022   Prob (F-statistic):               0.00
Time:                        16:47:54   Log-Likelihood:                -9880.2
No. Observations:                1019   AIC:                         1.980e+04
Df Residuals:                     999   BIC:                         1.990e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
price           

- 각 변수들의 p-value 값이 0.8, 0.6 이상이 되는 값들이 보이기에 다중공선성이 의심됩니다(통상 0.5이상이라면 다중공선성 확인). 또한 모델 summary 결과에서도 다중공선성(multicollinearity)이 의심된다는 결과가 나왔기에 VIF(분산팽창지수)를 확인해보았습니다. 

In [142]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif["features"] = X_train.columns
print(vif)

     VIF Factor                      features
0      4.663913                         price
1      1.154236                  retail_price
2      1.060875                uses_ad_boosts
3      1.250839                        rating
4           inf                  rating_count
5           inf             rating_five_count
6           inf             rating_four_count
7           inf            rating_three_count
8           inf              rating_two_count
9           inf              rating_one_count
10          inf                  badges_count
11          inf           badge_local_product
12          inf         badge_product_quality
13          inf           badge_fast_shipping
14     1.252304   product_variation_inventory
15     4.489941         shipping_option_price
16          NaN           shipping_is_express
17     1.054437          countries_shipped_to
18     1.037244               inventory_total
19     1.305460         merchant_rating_count
20     1.223057               merc

- vif값이 10이 넘으면 다중공선성을 의심합니다. 여기선 7개만을 제외하고 모두 10이 넘고, inf값까지 나온것을 볼 수 있습니다. 그렇기에 해당 값들을 drop하려 합니다.

### 변수 선택
- 변수선택법으로는 전진선택법을 선택하였습니다.
- 선택기준으로, AIC, BIC 등 중에서 실제값의 분포를 보려는 것이 아니라 예측을 하는 것이니 AIC를 선택하였습니다.
- 관련 링크 
  - https://aliencoder.tistory.com/24
  - https://zephyrus1111.tistory.com/65

In [143]:
# 변수선택을 위해서 타겟과 features가 함께 있는데이터로 구하자
select = features
select.append('units_sold')

df_for_select = df[select]

In [144]:
# ## 전진 선택법
# def forward(df, target):
#   variables = y_train ## 설명 변수 리스트
  
#   y = df[target] ## target
#   selected_variables = [] ## 선택된 변수들
#   sl_enter = 0.05 # 미리 설정한 a(최소 p-value)
  
#   sv_per_step = [] ## 각 스텝별로 선택된 변수들
#   adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
#   steps = [] ## 스텝
#   step = 0
#   while len(variables) > 0:
#       remainder = list(set(variables) - set(selected_variables))
#       pval = pd.Series(index=remainder) ## 변수의 p-value
#       ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
#       ## 선형 모형을 적합한다.
#       for col in remainder: 
#           X = df[selected_variables+[col]]
#           X = sm.add_constant(X)
#           model = sm.OLS(y,X).fit() # 모델
#           pval[col] = model.pvalues[col]
  
#       min_pval = pval.min()
#       if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
#           selected_variables.append(pval.idxmin())
          
#           step += 1
#           steps.append(step)
#           adj_r_squared = sm.OLS(y,sm.add_constant(df[selected_variables])).fit().rsquared_adj
#           adjusted_r_squared.append(adj_r_squared)
#           sv_per_step.append(selected_variables.copy())
#       else:
#           break
#   return selected_variables

In [145]:
variables = df.columns[:-2].tolist()

In [146]:
def forward_feature_selection(X_train, y_train, variables=X_train.columns.tolist() ):
    import statsmodels.api as sm
    import matplotlib.pyplot as plt
    import warnings
    warnings.filterwarnings("ignore")
    
    y = y_train ## 반응 변수

    selected_variables = [] ## 선택된 변수들
    sl_enter = 0.05
    sl_remove = 0.05
    
    sv_per_step = [] ## 각 스텝별로 선택된 변수들
    adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
    steps = [] ## 스텝
    step = 0
    while len(variables) > 0:
        remainder = list(set(variables) - set(selected_variables))
        pval = pd.Series(index=remainder) ## 변수의 p-value
        ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
        ## 선형 모형을 적합한다.
        for col in remainder: 
            X = X_train[selected_variables+[col]]
            X = sm.add_constant(X)
            model = sm.OLS(y,X).fit(disp=0)
            pval[col] = model.pvalues[col]
    
        min_pval = pval.min()
        if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
            selected_variables.append(pval.idxmin())
            ## 선택된 변수들에대해서
            ## 어떤 변수를 제거할지 고른다.
            while len(selected_variables) > 0:
                selected_X = X_train[selected_variables]
                selected_X = sm.add_constant(selected_X)
                selected_pval = sm.OLS(y,selected_X).fit(disp=0).pvalues[1:] ## 절편항의 p-value는 뺀다
                max_pval = selected_pval.max()
                if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                    remove_variable = selected_pval.idxmax()
                    selected_variables.remove(remove_variable)
                else:
                    break
            
            step += 1
            steps.append(step)
            adj_r_squared = sm.OLS(y,sm.add_constant(X_train[selected_variables])).fit(disp=0).rsquared_adj
            adjusted_r_squared.append(adj_r_squared)
            sv_per_step.append(selected_variables.copy())
        else:
            break
    return selected_variables

In [147]:
selected_variables = forward_feature_selection(X_train, y_train)

In [148]:
selected_variables

['rating_five_count',
 'rating_three_count',
 'const',
 'merchant_rating_count',
 'rating_four_count']

In [149]:
model_2 = sm.OLS(y_train, sm.add_constant(pd.DataFrame(X_train[selected_variables]))).fit(disp=0)
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:             units_sold   R-squared:                       0.842
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     1352.
Date:                Mon, 04 Jul 2022   Prob (F-statistic):               0.00
Time:                        16:47:55   Log-Likelihood:                -9885.2
No. Observations:                1019   AIC:                         1.978e+04
Df Residuals:                    1014   BIC:                         1.981e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
rating_five_count         4.03

### 할 일
- 과대적합, 과소적합 확인 후 릿지, 라쏘 사용 이유 찾기
- 이후 릿지 라쏘 적용.